In [96]:
import nltk
import unidecode
import pandas as pd
import os, re, string, gzip, itertools
import numpy as np
import tensorflow as tf
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from keras import utils
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from collections import defaultdict
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
from keras.layers import Dense, LSTM, Embedding, Activation, Dropout
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer,text_to_word_sequence
from keras.preprocessing import text, sequence
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
print("You have TensorFlow version", tf.__version__)

You have TensorFlow version 2.5.0


In [8]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [9]:
def calc_accuracy_score(clf, train_data, train_target, test_data, test_target, gs = True):
    clf_ = clf.fit(train_data, train_target)
    predicted_test_target = clf_.predict(test_data)
    predicted_train_target = clf_.predict(train_data)
    if True:
        print('For {} classifier accuracy on train data is {}'.format(clf_.steps[-1][-1], accuracy_score(train_target,predicted_train_target)))
        print('For {} classifier accuracy on test data is {}'.format(clf_.steps[-1][-1], accuracy_score(test_target, predicted_test_target)))
    else:
        print('For {} classifier accuracy on train data is {}'.format(clf_, accuracy_score(train_target,predicted_train_target)))
        print('For {} classifier accuracy on test data is {}'.format(clf_, accuracy_score(test_target, predicted_test_target)))
        

In [10]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words =set(stopwords.words('english'))


[nltk_data] Error loading stopwords: <urlopen error [Errno 8] nodename
[nltk_data]     nor servname provided, or not known>


In [11]:
twenty_train = fetch_20newsgroups(subset='train', shuffle=True)
twenty_test = fetch_20newsgroups(subset='test', shuffle=True)

In [12]:
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf_svm = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42))])
text_clf_stop = Pipeline([('vect', CountVectorizer(stop_words='english')), ('tfidf', TfidfTransformer()), 
                     ('clf', MultinomialNB())])
SVC_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
LogReg_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(LogisticRegression(solver='sag'), n_jobs=1)),
            ])

NB_pipeline = Pipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])


classfiers = [text_clf, text_clf_svm, text_clf_stop, SVC_pipeline, LogReg_pipeline, NB_pipeline]
parameters = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}
parameters_svm = {'vect__ngram_range': [(1, 1), (1, 2)], 'tfidf__use_idf': (True, False),'clf-svm__alpha': (1e-2, 1e-3)}

In [1]:
# count =0
# for classfier in classfiers:
#     count+=1
#     print("Model : {} ".format(count))
#     calc_accuracy_score(classfier, twenty_train.data, twenty_train.target, twenty_test.data, twenty_test.target)
#     print("\n")
    

In [111]:
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)

gs_clf = gs_clf.fit(twenty_train.data, twenty_train.target)
gs_clf.best_score_
gs_clf.best_params_
gs_clf_svm = GridSearchCV(text_clf_svm, parameters_svm, n_jobs=-1)
gs_clf_svm = gs_clf_svm.fit(twenty_train.data, twenty_train.target)
gs_clf_svm.best_score_
gs_clf_svm.best_params_

{'clf-svm__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}

In [2]:
# count =0
# for classfier in [gs_clf, gs_clf_svm]:
#     count+=1
#     print("Model : {} ".format(count))
#     calc_accuracy_score(classfier, twenty_train.data, twenty_train.target, twenty_test.data, twenty_test.target, gs=False)
#     print("\n")

In [ ]:
df = pd.read_csv('jigsaw-toxic-comment-classification-challenge/train.csv')

In [ ]:
def preprocess(text):
  	# Create Doc object
    doc = nlp(text, disable=['ner', 'parser'])
    # Generate lemmas
    lemmas = [token.lemma_ for token in doc]
    # Remove stopwords and non-alphabetic characters
    a_lemmas = [lemma for lemma in lemmas 
            if lemma.isalpha() and lemma not in stop_words]
    
    return ' '.join(a_lemmas)


In [1]:
ls

Text+Classification+using+python,+scikit+and+nltk.ipynb
Text+Classification+using+python,+scikit+and+nltk.py
Text_Classification.ipynb
classfication.ipynb
consumer_complaints.csv


In [4]:
df = pd.read_csv('Consumer_Complaints.csv', encoding='latin-1')


/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [ ]:
df['Product'].value_counts()

In [ ]:
# Split data into train and test
train_size = int(len(df) * .8)
print ("Train size: %d" % train_size)
print ("Test size: %d" % (len(df) - train_size))

In [ ]:
train_narrative = twen
train_product = df['Product'][:train_size]

test_narrative = df['Consumer complaint narrative'][train_size:]
test_product = df['Product'][train_size:]

In [13]:
train_narrative = twenty_train.data
train_product = twenty_train.target

test_narrative = twenty_test.data
test_product = twenty_test.target

In [14]:
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_narrative) # only fit on train
x_train = tokenize.texts_to_matrix(train_narrative)
x_test = tokenize.texts_to_matrix(test_narrative)

In [15]:
# Use sklearn utility to convert label strings to numbered index
encoder = LabelEncoder()
encoder.fit(train_product)
y_train = encoder.transform(train_product)
y_test = encoder.transform(test_product)

In [18]:
# Converts the labels to a one-hot representation
num_classes = np.max(y_train) + 1
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

In [19]:
# Inspect the dimenstions of our training and test data (this is helpful to debug)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

x_train shape: (11314, 1000)
x_test shape: (7532, 1000)
y_train shape: (11314, 20)
y_test shape: (7532, 20)


In [20]:
batch_size = 32
epochs = 5

In [21]:
# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

2021-08-13 23:34:45.057190: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [22]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

2021-08-13 23:34:49.597499: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/5
319/319 [==============================] - 19s 7ms/step - loss: 2.2704 - accuracy: 0.3393 - val_loss: 0.9573 - val_accuracy: 0.7314
Epoch 2/5
319/319 [==============================] - 1s 4ms/step - loss: 0.8193 - accuracy: 0.7710 - val_loss: 0.7684 - val_accuracy: 0.7721
Epoch 3/5
319/319 [==============================] - 1s 4ms/step - loss: 0.5417 - accuracy: 0.8453 - val_loss: 0.7313 - val_accuracy: 0.7836
Epoch 4/5
319/319 [==============================] - 1s 4ms/step - loss: 0.4035 - accuracy: 0.8877 - val_loss: 0.7382 - val_accuracy: 0.7792
Epoch 5/5
319/319 [==============================] - 1s 4ms/step - loss: 0.3181 - accuracy: 0.9146 - val_loss: 0.7332 - val_accuracy: 0.7959


In [23]:
# Evaluate the accuracy of our trained model
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

236/236 [==============================] - 1s 2ms/step - loss: 1.2025 - accuracy: 0.6668
Test score: 1.2024712562561035
Test accuracy: 0.666755199432373


In [36]:
# # Here's how to generate a prediction on individual examples
# text_labels = encoder.classes_ 

# for i in range(10):
#     prediction = model.predict(np.array([x_test[i]]))
#     predicted_label = text_labels[np.argmax(prediction)]
#     print(test_narrative.iloc[i][:50], "...")
#     print('Actual label:' + test_product.iloc[i])
#     print("Predicted label: " + predicted_label + "\n")

In [119]:
lemma = WordNetLemmatizer()
def clean_text(text):
    text = re.sub(pattern=r'<.*?>', repl=' ', string=text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()
    text="".join([i for i in text if i not in string.punctuation])
    text = unidecode.unidecode(text)
    text = text.strip()
  
    return text



In [120]:
text = 'Hello 123   <br> hghjh</br> completed implementation dèèp lèarning ánd cömputer vísíön satyam@gmail.com , p>>..>>>#9 YUJ hsddu gh ssd [[23/23/344]] https:www.google.com js'

In [121]:
clean_text(text)

'hello      hghjh  completed implementation deep learning and computer vision satyamgmailcom  p yuj hsddu gh ssd   js'